In [1]:
import pandas as pd
import pyodbc
import numpy as np
import os

In [22]:
path = r'T:\Tableau\tableauJourneyToWork\Datasources'

In [73]:
# adjust accordingly on the tables ModeShare_ALL_Years; ModeByVehiclesAvailable_AllYears; 
# ModeByPovertyStatus_AllYears; TimeLeavingForWork_AllYears; TravelTimeToWork_AllYears
data = pd.read_csv(path + '\\ModeShare_ALL_Years.csv')   
df = pd.DataFrame(data)

print(df)

     Year             Geography                              Mode  Estimate  \
0    2009                Coburg                           Bicycle         2   
1    2009                Coburg    Car, truck, or van - Carpooled        83   
2    2009                Coburg  Car, truck, or van - Drove alone       373   
3    2009                Coburg                        Motorcycle         0   
4    2009                Coburg                       Other means         3   
..    ...                   ...                               ...       ...   
580  2021  Salem Urbanized Area                        Motorcycle       212   
581  2021  Salem Urbanized Area                           Bicycle      1248   
582  2021  Salem Urbanized Area                            Walked      3067   
583  2021  Salem Urbanized Area                       Other means       782   
584  2021  Salem Urbanized Area                    Worked at home     11010   

     MOE_Est     Share  MOE_Share   SharePct  MOE_S

In [74]:
df.dtypes

Year              int64
Geography        object
Mode             object
Estimate          int64
MOE_Est           int64
Share           float64
MOE_Share       float64
SharePct        float64
MOE_SharePct    float64
dtype: object

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=rliddb.int.lcog.org,5433;'
                      'Database=GIS_CLMPO;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [66]:
df['Length.of.Commute'].unique()

array(['Less than 5 minutes', '5 to 9 minutes', '10 to 14 minutes',
       '15 to 19 minutes', '20 to 24 minutes', '25 to 29 minutes',
       '30 to 34 minutes', '35 to 39 minutes', '40 to 44 minutes',
       '45 to 59 minutes', '60 to 89 minutes', '90 or more minutes'],
      dtype=object)

In [67]:
len('90 or more minutes')

18

In [75]:
df.replace({np.inf: np.nan, -np.inf: np.nan}, inplace=True)

In [76]:
df = df.fillna(0)

In [77]:
# table names: mode_share, mode_by_vehicles_available, mode_by_poverty_status, time_leaving_for_work, travel_time_to_work
# rewrite tables: DROP TABLE mode_share;
# edit accordingly the table
cursor.execute('''
            CREATE TABLE mode_share_test(
            year int,
            geo varchar(50),
            mode varchar(50),
            estimate int,
            moe_est int,
            share float,
            moe_share float,
            share_pct float,
            moe_share_pct float
            )
               ''')

In [71]:
#df.rename(columns={'Mode.by.Vehicle.Availability': 'Mode', 'Vehicles.Available': 'VHCL_AVAIL'}, inplace=True)
#df.rename(columns={'Travel.Mode': 'Mode', 'Poverty.Status': 'Poverty_Status'}, inplace=True)
#df.rename(columns={'Time.Leaving.for.Work..Census.': 'Time_Leaving', 'Time.Leaving.for.Work': 'FormattedTime'}, inplace=True)
df.rename(columns={'Length.of.Commute': 'length_of_commute'}, inplace=True)

In [78]:
ndf = df[df.Year != 2021]

In [80]:
# mode_share, mode_by_vehicles_available, mode_by_poverty_status, time_leaving_for_work, travel_time_to_work
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO mode_share_test(year, geo, mode, estimate, moe_est, share, moe_share, share_pct, moe_share_pct)
                VALUES (?,?,?,?,?,?,?,?,?)
                ''',
                   row.Year,
                   row.Geography,
                   row.Mode,
                   row.Estimate,
                   row.MOE_Est,
                   row.Share,
                   row.MOE_Share,
                   row.SharePct,
                   row.MOE_SharePct
                  )
conn.commit()